In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# 1. Define the Ticker
ticker_symbol = "SPY"
ticker = yf.Ticker(ticker_symbol)

# 2. Get all available expiration dates
expirations = ticker.options
print(f"Found {len(expirations)} expiration dates. First 5: {expirations[:5]}")

# 3. Fetch the Chain for ALL expirations (This might take 30-60 seconds)
options_data = []

print("Fetching options chains... (This may take a moment)")

for date in expirations:
    try:
        # Get the chain for this specific date
        opt = ticker.option_chain(date)
        calls = opt.calls
        
        # We need to track the 'Time to Expiration' (T)
        # Calculate days until expiration
        exp_date = datetime.strptime(date, "%Y-%m-%d")
        days_to_exp = (exp_date - datetime.now()).days
        
        # Filter for reasonable data (Liquid options only)
        # We skip options with 0 volume to avoid bad data
        calls = calls[calls['volume'] > 0].copy()
        
        # Add metadata
        calls['expirationDate'] = date
        calls['daysToExpiration'] = days_to_exp
        
        options_data.append(calls)
    except Exception as e:
        print(f"Skipping {date}: {e}")

# 4. Combine into one DataFrame
df_options = pd.concat(options_data)

# 5. Clean Up
# We need Strike, Price (LastPrice or Bid/Ask average), and Time
df_options['mid_price'] = (df_options['bid'] + df_options['ask']) / 2
# If bid/ask is missing/zero, fallback to lastPrice
df_options['price'] = np.where(df_options['mid_price'] > 0, df_options['mid_price'], df_options['lastPrice'])

print(f"Data Fetch Complete. Loaded {len(df_options)} call options.")
print(df_options[['strike', 'price', 'daysToExpiration', 'impliedVolatility']].head())

Found 26 expiration dates. First 5: ('2026-01-05', '2026-01-06', '2026-01-07', '2026-01-08', '2026-01-09')
Fetching options chains... (This may take a moment)
Data Fetch Complete. Loaded 3383 call options.
   strike   price  daysToExpiration  impliedVolatility
2   615.0  68.480                 2           0.585942
3   630.0  53.485                 2           0.672367
4   640.0  43.490                 2           0.573491
8   655.0  28.480                 2           0.415533
9   660.0  23.480                 2           0.362433


In [2]:
import scipy.stats as si

# 1. Get Global Inputs (Spot Price & Risk-Free Rate)
# We use the 10-Year Treasury Yield (^TNX) as a proxy for the risk-free rate
try:
    r = yf.Ticker("^TNX").history(period="1d")['Close'].iloc[-1] / 100
except:
    r = 0.045 # Fallback to 4.5% if data fetch fails

# Get current SPY price (Spot Price 'S')
S = ticker.history(period="1d")['Close'].iloc[-1]

print(f"Global Inputs -> Spot Price (S): ${S:.2f}, Risk-Free Rate (r): {r:.2%}")

# 2. Define Helper Functions (The Math)

def norm_pdf(x):
    """Standard Normal Probability Density Function"""
    return (1.0 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * x**2)

def norm_cdf(x):
    """Standard Normal Cumulative Distribution Function"""
    return si.norm.cdf(x, 0.0, 1.0)

def calculate_d1_d2(S, K, T, r, sigma):
    """Calculates d1 and d2 for Black-Scholes"""
    # Safety check for sigma to avoid divide by zero
    if sigma <= 0: return 0, 0
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2

def black_scholes_call(S, K, T, r, sigma):
    """Calculate theoretical Call Price"""
    d1, d2 = calculate_d1_d2(S, K, T, r, sigma)
    return S * norm_cdf(d1) - K * np.exp(-r * T) * norm_cdf(d2)

def calculate_vega(S, K, T, r, sigma):
    """Calculate Vega (Derivative of Price w.r.t Volatility)"""
    d1, _ = calculate_d1_d2(S, K, T, r, sigma)
    return S * norm_pdf(d1) * np.sqrt(T)

# 3. Newton-Raphson Solver
def get_implied_volatility(market_price, S, K, T, r):
    """
    Solves for sigma using Newton-Raphson.
    """
    # Max iterations and precision tolerance
    MAX_ITER = 100
    PRECISION = 1e-5
    
    # Initial Guess (e.g., 50% vol)
    sigma = 0.5
    
    for i in range(MAX_ITER):
        # Calculate Model Price and Vega with current sigma
        price = black_scholes_call(S, K, T, r, sigma)
        vega = calculate_vega(S, K, T, r, sigma)
        
        # Calculate the difference (Price Error)
        diff = market_price - price
        
        # Check for convergence
        if abs(diff) < PRECISION:
            return sigma
        
        # If Vega is too small, the update step will explode (Divide by zero risk)
        if abs(vega) < 1e-8:
            return np.nan
        
        # Update sigma (Newton-Raphson Step)
        sigma = sigma + diff / vega
        
    # If loop finishes without convergence
    return np.nan

print("Math functions defined. Ready to compute.")

Global Inputs -> Spot Price (S): $683.17, Risk-Free Rate (r): 4.19%
Math functions defined. Ready to compute.


In [3]:
# 1. Convert Days to Years (T)
# Black-Scholes requires Time in Years, not Days
df_options['T'] = df_options['daysToExpiration'] / 365.0

# Avoid division by zero for options expiring today
df_options = df_options[df_options['T'] > 0.001].copy()

print("Calculating Implied Volatility for 3,000+ options... (This uses your Newton-Raphson solver)")

# 2. Apply the Solver
# We use a lambda function to pass the current row's data + our global S and r variables
df_options['Calculated_IV'] = df_options.apply(
    lambda row: get_implied_volatility(
        row['price'],
        S,
        row['strike'],
        row['T'],
        r
    ), axis=1
)

# 3. Clean the Data
# Remove failures (NaN) and unrealistic outliers (IV > 300% or IV < 1%)
df_clean = df_options.dropna(subset=['Calculated_IV']).copy()
df_clean = df_clean[(df_clean['Calculated_IV'] > 0.01) & (df_clean['Calculated_IV'] < 3.0)]

print(f"Calculation Complete. Successfully solved {len(df_clean)} contracts.")
print(df_clean[['strike', 'expirationDate', 'price', 'Calculated_IV']].head())

Calculating Implied Volatility for 3,000+ options... (This uses your Newton-Raphson solver)
Calculation Complete. Successfully solved 2918 contracts.
   strike expirationDate   price  Calculated_IV
2   615.0     2026-01-05  68.480       0.656719
3   630.0     2026-01-05  53.485       0.525638
4   640.0     2026-01-05  43.490       0.438360
8   655.0     2026-01-05  28.480       0.299478
9   660.0     2026-01-05  23.480       0.253663


In [4]:
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install nbformat --upgrade

Note: you may need to restart the kernel to use updated packages.


In [6]:
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata

# 1. Prepare Data for 3D Plotting
# We need grid coordinates (X, Y) and values (Z)
x = df_clean['strike']
y = df_clean['T']  # Time to Expiration (Years)
z = df_clean['Calculated_IV']

# 2. Create a Mesh Grid (Interpolation)
# This creates a smooth surface between your data points
xi = np.linspace(x.min(), x.max(), 50)
yi = np.linspace(y.min(), y.max(), 50)
X, Y = np.meshgrid(xi, yi)

# Griddata fills in the gaps between actual strike prices
Z = griddata((x, y), z, (X, Y), method='linear')

# 3. Plot with Plotly
fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Viridis')])

# 4. Make it pretty
fig.update_layout(
    title='SPY Implied Volatility Surface (Newton-Raphson Solver)',
    scene = dict(
        xaxis_title='Strike Price ($)',
        yaxis_title='Time to Expiration (Years)',
        zaxis_title='Implied Volatility (IV)',
        aspectratio=dict(x=1, y=1, z=0.5) # Flattens the Z-axis slightly for better view
    ),
    width=900,
    height=700
)

fig.show()